In [1]:
import os 


In [2]:
%pwd

'n:\\Users\\Farshu\\Projects\\Data-Science-Project-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'n:\\Users\\Farshu\\Projects\\Data-Science-Project-with-MLFlow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from pathlib import Path
from mlProject.utils.common import read_yaml, create_directories
from mlProject.constants import *

# Define the file paths before using them
CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")

# Configuration Manager class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        """Reads the YAML config files and creates necessary directories."""
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Ensure artifacts directory exists
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Fetches data ingestion config and creates required directories."""
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )


In [8]:
import os
import urllib.request as request
import zipfile
from pathlib import Path
from mlProject import logger
from mlProject.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """Downloads the dataset if not already present."""
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded successfully! Headers:\n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """Extracts the dataset zip file into the specified directory."""
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        logger.info(f"Extracting {self.config.local_data_file} to {unzip_path}...")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extraction complete: Files extracted to {unzip_path}")


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-27 22:22:48,868: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-03-27 22:22:48,889: INFO: common: YAML file: params.yaml loaded successfully]
[2025-03-27 22:22:48,893: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-03-27 22:22:48,897: INFO: common: Created directory at: artifacts]
[2025-03-27 22:22:48,901: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-03-27 22:22:50,326: INFO: 2780294285: artifacts/data_ingestion/data.zip downloaded successfully! Headers:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 44B3:6EE3D:11E33C:249B4F:67E58262
Accept-Ranges: bytes
Date: Thu, 2